# Short Lab 3 : Proximal/cyclic/greedy coordinate descent

#### Authors: A. Gramfort, H. Janati, M. Massias

## Aim

The aim of this material is to code 
- cyclic and greedy coordinate descent for ordinary least squares (OLS)
- proximal coordinate descent for sparse Logistic regression

## VERY IMPORTANT

- This work **must be done by pairs of students**.
- **Each** student must send their work **before the 22nd of november at noon**, using the **moodle platform**.
- This means that **each student in the pair sends the same file**
- The **name of the file must be** constructed as in the next cell

# Gentle reminder: no evaluation if you don't respect this EXACTLY

### How to construct the name of your file

In [2]:
# Change here using YOUR first and last names
fn1 = "nicolas"
ln1 = "saint"
fn2 = "matthis"
ln2 = "guerin"

filename = "_".join(map(lambda s: s.strip().lower(),
                        ["lab3", ln1, fn1, "and", ln2, fn2])) + ".ipynb"
print(filename)

lab3_saint_nicolas_and_guerin_matthis.ipynb


In [1]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt

In [2]:
# the usual functions:

from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import randn


def simu(coefs, n_samples=1000, corr=0.5, for_logreg=False):
    n_features = len(coefs)
    cov = toeplitz(corr ** np.arange(0, n_features))
    A = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    b = A.dot(coefs) + randn(n_samples)
    if for_logreg:
        b = np.sign(b)
    return A, b

/tmp/ipykernel_5938/428856815.py:4: DeprecationWarning: Please use `toeplitz` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import toeplitz


## Part 1: Ordinary Least Squares


Let $A \in \mathbb{R}^{n \times p}$, $y \in \mathbb{R}^n$.
We want to use coordinate descent to solve:
    $$\hat w \in  \mathrm{arg \, min \,} \frac 12 \Vert Aw - b \Vert ^2 $$


<div class="alert alert-success">
    <b>QUESTION 1:</b> We ask you to code
     <ul>
         <li>cyclic coordinate descent: at iteration $t$, update feature $j = t \mod p$</li>
         <li>greedy coordinate descent: at iteration $t$, update feature having the largest partial gradient in magnitude, ie $j = \mathrm{arg\, max \,}_{i} \vert \nabla_i f(w_t) \vert$.
</li>
    </ul>
</div>

**WARNING**: You must do this in a clever way, ie such that $p$ updates cost the same as one update of GD.

In [3]:
n_features = 100
np.random.seed(1970)
coefs = np.random.randn(n_features)

A, b = simu(coefs, n_samples=1000, for_logreg=False)

In [4]:
def cyclic_cd(A, b, n_iter):
    n_samples, n_features = A.shape
    all_objs = []

    x_hat = np.zeros(n_features)
    residuals = b - A.dot(x_hat)

    for t in range(n_iter):
        i = t % n_features
        old_x_i = x_hat[i]
        x_hat[i] += A[:, i].dot(residuals) / np.sum(A[:, i] ** 2)
        residuals += A[:, i] * (old_x_i - x_hat[i])

        if t % n_features == 0:
            all_objs.append(np.sum(residuals ** 2) / 2.)

    return x_hat, np.array(all_objs)


def greedy_cd(A, b, n_iter):
    n_samples, n_features = A.shape
    all_objs = []
    
    w = np.zeros(n_features)
    
    gradient = A.T.dot(A.dot(w) - b)
    gram = A.T.dot(A)  # you will need this to keep the gradient up to date
    
    # TODO
    lips_const = np.linalg.norm(A, axis=0) ** 2
    # END TODO 
    
    for t in range(n_iter):
        # TODO
        # choose feature j to update:
        j = np.argmax(np.abs(gradient))
        old_w_j = w[j]
        w[j] -= gradient[j] / lips_const[j]
        # update gradient:
        gradient -= gram[:, j] * (old_w_j - w[j])
        # END TODO

        if t % n_features == 0:
            all_objs.append(0.5 * np.linalg.norm(A.dot(w) - b) ** 2)
    
    return w, np.array(all_objs)

<div class="alert alert-success">
    <b>QUESTION 2:</b>
     <ul>
         <li>Compute a precise minimum with your favorite solver</li>
         <li>Compare the performance of cyclic and greedy CD as function of iterations.</li>
         <li>From a practical point of view, could you use greedy CD for L2 regularized logistic regression? to solve OLS, but with 100,000 features? Explain your answers.</li>
    </ul>
</div>

**Remark:** You will do the plots using the number of iterations on the x-axis and not time as your code is likely to be slow unless you use [numba](https://numba.pydata.org/).

In [5]:
# Compute a precise minimum with scipy.optimize.fmin_l_bfgs_b

from scipy.optimize import fmin_l_bfgs_b

def loss(w):
    return norm(A.dot(w) - b) ** 2 / 2.

def grad(w):
    return A.T.dot(A.dot(w) - b)

w_min, _, _ = fmin_l_bfgs_b(loss, np.zeros(n_features), grad, pgtol=1e-30, factr=1e-30)


In [6]:
# Compare the performance of cyclic and greedy CD as function of iterations with semylogy

n_iter = 10000
all_objs = {}
for name, method in [("cyclic", cyclic_cd), ("greedy", greedy_cd)]:
    _, all_objs[name] = method(A, b, n_iter)

plt.figure(figsize=(15, 5))
for name in all_objs:
    plt.semilogy((all_objs[name] - w_min)).clip(1e-30), label=name)
plt.legend(loc="best")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Convergence plot for cyclic and greedy CD")

plt.show()

SyntaxError: cannot assign to function call (3549423389.py, line 10)

## Part 2: Sparse Logistic regression

### An important result

Remember: we are solving 
$$\hat w \in \mathrm{arg \, min} \sum_{i=1}^{n} \mathrm{log} ( 1 + e^{- y_i w^\top x_i} )  + \lambda \Vert w \Vert_1$$

<div class="alert alert-success">
    <b>QUESTION 3:</b><br/>
    Assuming uniqueness of the solution, show that: $\lambda \geq \lambda_{max} \Leftrightarrow \hat w = 0$
where $\lambda_{max} := \frac 12 \Vert X^\top y \Vert_{\infty}$.
</div>

**HINT:** You will need the following beautiful result: for any $w =(w_1, \dots, w_p) \in \mathbb{R}^p$, the subdifferential of the L1 norm at $w$ is:

$$
\partial \Vert \cdot \Vert_1 (w) = \partial \vert \cdot \vert (w_1)  \times \dots \times \partial \vert \cdot \vert (w_p) $$
where $\times$ is the Cartesian product between sets,
and $$ \partial \vert \cdot \vert (w_j) = 
\begin{cases} &w_j / |w_j| &\mathrm{if} \quad w_j \neq 0, 
         \\ & [-1, 1] &\mathrm{otherwise.} 
\end{cases}
$$


(it should now be easy to find $\partial \Vert \cdot \Vert_1 (\mathbf{0}_p)$)

*answer here*

<div class="alert alert-success">
    <b>QUESTION 4:</b><br/>
    Show that for sparse Logistic regression the coordinate-wise Lipschitz constant of the smooth term, $\gamma_j$, can be taken equal to $\Vert X_j \Vert^2 / 4$, where $X_j$ denotes the $j$-th column of $X$.
</div>

% Adjusted answer starts here

By a theorem, we can establish that a twice-differentiable function, which is \(L\)-smooth, is bounded by \(L\):

\[
\text{F is } L\text{-Smooth and twice-differentiable} \iff \|\nabla^2F(w)\|_2 \leq L
\]

Now, let's compute the second derivatives:

Let \(\sigma(x) = \frac{1}{1+\mathrm{e}^{-x}}\) for simplification.

\[
\begin{align*}
\dfrac{\partial^2}{\partial w_j^2} F(w_j) &= \sum_{i=1}^n\dfrac{\partial^2}{\partial w_j^2}\log \big( 1 + e^{- y_i w^\top x_i} \big) \\
&= \sum_{i=1}^n \dfrac{\partial}{\partial w_j} \Big(-y_i x_{i,j}\big(\sigma(- y_i w^\top x_{i}) \big) \Big) \\
&= \sum_{i=1}^n (-y_i x_{i,j})^2\sigma'(y_i w^\top x_{i})
\end{align*}
\]

We know that \(\sigma'(x) = \sigma(x)\big(1-\sigma(x)\big) \leq \frac{1}{4}\). 

Moreover, for logistic regression, we have \(y_i \in \{-1, +1\}\) and \(y_i^2 = 1\).

\[
\begin{align*}
\dfrac{\partial^2}{\partial w_j^2} F(w_j) &\leq \frac{1}{4} \sum_{i=1}^n (y_i x_{i,j})^2 \\
&\leq \frac{1}{4} \sum_{i=1}^n x^2_{i,j} \\
&\leq \frac{1}{4} \|X_j\|_2^2 \quad \text{with } X_j \in \mathbb{R}^p
\end{align*}
\]

So, we can express:

\[
\boxed{\gamma_j = L_j = \frac{1}{4} \|X_j\|_2^2}
\]


<div class="alert alert-success">
    <b>QUESTION 5:</b><br/>
    Code cyclic proximal coordinate descent for sparse Logistic regression:
</div>

**WARNING**: the Lasso means linear regression (quadratic fitting term) with L1 penalty. Sparse logistic regression means logistic regression with L1 penalty.

In [7]:
X, y = simu(coefs, n_samples=1000, for_logreg=True)
lambda_max = norm(X.T.dot(y), ord= np.inf) / 2.
lamb = lambda_max / 20.  
# much easier to parametrize lambda as a function of lambda_max than 
# to take random values like 0.1 in previous Labs


def sigmoid(t):
    """Sigmoid function"""
    return 1. / (1. + np.exp(-t))


def soft_thresh(x, u):
    """Soft thresholding of x at level u"""
    return np.sign(x) * np.maximum(0., np.abs(x) - u)


def cd_logreg(X, y, lamb, n_iter):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    Xw = X.dot(w)
    
    # TODO
    lips_const = np.linalg.norm(X, axis=0) ** 2 / 4.
    # END TODO
    
    for t in range(n_iter):
        for j in range(n_features):
            old_w_j = w[j]
            # TODO
            grad_j = -y * sigmoid(-y * Xw)
            w[j] = soft_thresh(1, 2)
            
            if old_w_j != w[j]:
                Xw += X[:, j] * (old_w_j - w[j])
            #END TODO
            
        all_objs[t] = np.log(1. + np.exp(-y * Xw)).sum() + lamb * norm(w, ord=1)
    
    return w, all_objs

# Part 3: Real data

We will compare vanilla cyclic CD and ISTA to solve the Lasso on a real dataset, called _leukemia_.

In [8]:
from sklearn.datasets import fetch_openml

leuk = fetch_openml("leukemia")

X = np.asfortranarray(leuk.data)
y = np.ones(leuk.target.shape)
y[leuk.target == leuk.target[0]] = -1.

/home/nico/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name leukemia exist. Versions may be fundamentally different, returning version 1.
  warn(


In [9]:
print(X.shape)

lambda_max_lasso = norm(X.T.dot(y), ord=np.inf)
lambd = lambda_max_lasso / 5.

(72, 7129)


<div class="alert alert-success">
    <b>QUESTION 6:</b> Code
    <ul>
        <li>a simple proximal gradient solver for the Lasso</li>
        <li>a prox CD solver for the Lasso and compare them on this dataset.</li>
    </ul>
</div>

**Remark:** Do the plots in terms of epochs, not updates (to be fair to CD).

In [14]:
def grad_linreg(w, X, y, lamb):
    return X.T.dot(X.dot(w) - y)

def loss_linreg(w, X, y, lamb):
    return np.linalg.norm(X @ w - y) ** 2 + lamb * norm(w, ord=1)

def PGD(X, y, loss_f, grad_f, lbd, n_iter):
    """Proximal gradient descent algorithm"""
    n_samples, n_features = X.shape
    w = np.random.randint(n_features) * 0.01
    objectives = []
    step = n_samples / ((norm(X, ord=2)**2))

    objectives.append(loss_f(w, X, y, lbd))
    for k in range(n_iter + 1):
        w = soft_thresh(w - step * grad_f(w, X, y, lbd), lbd * step)
        # Current objective
        objectives.append(loss_f(w, X, y, lbd))

    return w, np.array(objectives)

def cd_linreg(X, y, loss_f, lamb, n_iter):
    n_samples, n_features = X.shape
    w = np.random.randint(n_features) * 0.01
    Xw = X.dot(w)
    grad = X.T @ (Xw - y)
    # TODO
    step =  n_samples / [norm(X[:, j], ord=2)**2 for j in range(n_features)]
    gram = X.T @ X
    all_objs = np.zeros(n_iter+1)
    all_objs[0] = loss_f(w, X, y, lamb)
    # END TODO
    for t in range(n_iter):
        for j in range(n_features):
            old_w_j = w[j]
            grad_j = grad[j]
            w[j] = soft_thresh(old_w_j - grad_j * step[j], lamb * step[j])

            if old_w_j != w[j]:
                grad += (w[j] - old_w_j) * gram[:,j]
                Xw += X[:, j] * (w[j] - old_w_j) 
                
        all_objs[t+1] = loss_f(w, X, y, lamb)
    
    return w, all_objs

In [15]:
n_iter = 15000
# PGD Logistic Regression
pgd_x_leuk, pgd_x_list_leuk = PGD(X, y, loss_linreg, grad_linreg, lbd=lambd, n_iter=n_iter)
# Proximal Coordinate Descent Logistic Regression
proximal_x_leuk, proximal_x_list_leuk = cd_linreg(X, y, loss_linreg, lambd, n_iter)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)